In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import seaborn as sns
from itertools import product
from itertools import combinations_with_replacement
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time
import math
%matplotlib inline

In [111]:
from sklearn.cluster import KMeans, SpectralClustering, AffinityPropagation
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
from utils import *
%load_ext autoreload
%autoreload 2

In [4]:
gj = pd.read_csv('./gj_df.csv')
# gj = gj[['ifp_id', 'ctt', 'cond', 'training', 'team', 'user_id', 'value', 'fcast_date']]
# gj['fcast_year'] = pd.to_datetime(gj['fcast_date']).dt.year
# gj['fcast_week'] = pd.to_datetime(gj['fcast_date']).dt.week
# gj['ifp_week'] = gj['fcast_year'].map(str) + gj['fcast_week'].map(str) + gj['ifp_id']
# gj = gj.drop('fcast_date', axis=1)
# gj = gj.drop_duplicates()
# gj.to_csv('./gj_df.csv', index=False)

In [5]:
# adult = pd.read_csv('../labels.txt', delimiter='\t', header=0, names=['user_id','website','rating'])
# trec = pd.read_csv('../trec-rf10-crowd/trec-rf10-data.txt', delimiter='\t')
# gj = pd.read_csv('./filled_active_df.csv')

# best_users = trec.groupby('workerID').count().sort_values('docID', ascending=False)[:150].index
# trec = trec[trec['workerID'].isin(best_users)]

# r = pd.Series([2,3,2,3], index=[1,2,0,-2])
# trec['label_bin'] = trec['label'].map(r)
gj.head(10)

,ifp_id,ctt,cond,training,team,user_id,value,fcast_year,fcast_week,ifp_week
0,1244-0,1a,1,a,NaN,51,0.20,2015,3,201531244-0
1,1244-0,1a,1,a,NaN,51,0.20,2015,4,201541244-0
2,1244-0,1a,1,a,NaN,51,0.20,2015,5,201551244-0
3,1244-0,1a,1,a,NaN,51,0.20,2015,6,201561244-0
4,1244-0,1a,1,a,NaN,51,0.20,2015,7,201571244-0
5,1244-0,1a,1,a,NaN,51,0.20,2015,8,201581244-0
6,1244-0,1a,1,a,NaN,51,0.09,2015,8,201581244-0
7,1244-0,1a,1,a,NaN,51,0.09,2015,9,201591244-0
8,1394-0,1a,1,a,NaN,51,0.75,2014,42,2014421394-0
9,1394-0,1a,1,a,NaN,51,0.75,2014,43,2014431394-0


In [129]:
ifp_var = gj.groupby('ifp_week')['value'].var().sort_values()
ifps_less_var = (ifp_var<.1).index
ifps_less_var
testframe = testframe[testframe['ifp_week'].isin(ifps_less_var)]

In [130]:
# testframe = create_user_task_ids(adult, 'user_id', 'website', 'rating')
testframe = create_user_task_ids(gj, 'user_id', 'ifp_week', 'value', False, True)
testframe.head()

,ifp_id,ctt,cond,training,team,user_id,value,fcast_year,fcast_week,ifp_week,bin,task_id,uid,bin_levels
0,1244-0,1a,1,a,NaN,51,0.2,2015,3,201531244-0,0.2,0,0,2
1,1244-0,1a,1,a,NaN,51,0.2,2015,4,201541244-0,0.2,1,0,2
2,1244-0,1a,1,a,NaN,51,0.2,2015,5,201551244-0,0.2,2,0,2
3,1244-0,1a,1,a,NaN,51,0.2,2015,6,201561244-0,0.2,3,0,2
4,1244-0,1a,1,a,NaN,51,0.2,2015,7,201571244-0,0.2,4,0,2


In [151]:
# testframe['value'] = testframe['value'] - .5
testframe['value'] = testframe['value']*2

In [104]:
c = pd.cut(
    testframe['value'],
    [-np.inf, .2, .4, .6, .8, np.inf],
    labels=[2,3,5,7,11]
)
testframe['bin_levels'] = c

In [27]:
testframe = testframe[testframe['uid']<1000]

In [152]:
def batcher(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def split(df):
    train_df, validate_df, test_df = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    return train_df, validate_df, test_df

class Model(nn.Module):
    def __init__(self, users, tasks, k=2):
        super(Model, self).__init__()
        self.user_lut = nn.Embedding(users, k)
        self.task_lut = nn.Embedding(tasks, k)

        self.user_bias = nn.Embedding(users, 1)
        self.task_bias = nn.Embedding(tasks, 1)
        self.global_bias = nn.Parameter(torch.FloatTensor(1))
        
    def forward(self, users, jokes):
        user_vectors = self.user_lut(users)
        task_vectors = self.task_lut(jokes)
        user_bias = self.user_bias(users)
        task_bias = self.task_bias(jokes)

        return torch.bmm(user_vectors.unsqueeze(1),
                         task_vectors.unsqueeze(2)).squeeze() \
                         + user_bias.squeeze() + task_bias.squeeze() + self.global_bias.expand_as(user_bias.squeeze())
        
class Model2(nn.Module):
    def __init__(self, users, tasks, k=2):
        super(Model2, self).__init__()
        self.user_lut = nn.Embedding(users, k)
        self.task_lut = nn.Embedding(tasks, k)

        self.user_bias = nn.Embedding(users, 1)
        self.task_bias = nn.Embedding(tasks, 1)
        self.global_bias = nn.Parameter(torch.FloatTensor(1))
        
    def forward(self, users, jokes):
        user_vectors = self.user_lut(users)
        task_vectors = self.task_lut(jokes)
        user_bias = self.user_bias(users)
        task_bias = self.task_bias(jokes)

        preds = torch.bmm(user_vectors.unsqueeze(1), task_vectors.unsqueeze(2)).squeeze() \
                          + task_bias.squeeze() + self.global_bias.expand_as(user_bias.squeeze())
        m = nn.Tanh()
        return m((user_bias.squeeze() * (preds - .5)) + .5)
            

def val(df, model):
    crit = nn.MSELoss(size_average=False)
    crit2 = nn.L1Loss(size_average=False)
    total_loss = 0.
    total_l1 = 0.
    total_num = 0
    for batch in batcher(df, 100):
        true_rating = Variable(torch.Tensor(batch.bin.values.astype(float)))
        total_num = total_num + true_rating.size(0)
        users = Variable(torch.LongTensor(batch.uid.values))
        tasks = Variable(torch.LongTensor(batch.task_id.values))
        scores = model.forward(users, tasks)
        total_loss += crit(scores, true_rating).data[0]
        total_l1 += crit2(scores,true_rating).data[0]
    return math.sqrt(total_loss/total_num), total_l1/total_num


def train(train_iter, val_iter, test_iter, model):
    opt = optim.SGD(model.parameters(), lr=0.3)
    crit = nn.MSELoss()
    crit2 = nn.L1Loss()

    print("val:", val(validate_df, model))
    for epochs in range(30):
        avg_loss = 0
        avg_l1 = 0
        total = 0
        for i,batch in enumerate(batcher(train_df, 100)):
            opt.zero_grad()
            rating = Variable(torch.Tensor(batch.bin.values.astype(float)))
#             print(rating)
            users = Variable(torch.LongTensor(batch.uid.values))
            tasks = Variable(torch.LongTensor(batch.task_id.values))
            scores = model.forward(users, tasks) 
#             + torch.sum(model.user_lut.weight.data.pow(2)) + \
#             torch.sum(model.task_lut.weight.data.pow(2)) + torch.sum(model.user_bias.weight.data.pow(2)) + \
#             torch.sum(model.task_bias.weight.data.pow(2))
            loss = crit(scores, rating)
            #if i % 1000==0:
            #    print (loss.data[0])
            loss.backward()
            avg_loss += loss.data[0]
            avg_l1 += crit2(scores,rating).data[0]
            total += 1
            opt.step()
        print("train:", math.sqrt(avg_loss / float(total)), avg_l1/ float(total))
        print("val:", val(validate_df, model))
#         print(model.user_bias.weight.data)
    return model.user_lut.weight.data, model.user_bias.weight.data

In [153]:
train_df, validate_df, test_df = split(testframe)
users = len(train_df.uid.unique())
tasks = len(train_df.task_id.unique())
model = Model2(users, tasks, k=4)
user_vec, user_bias = train(train_df, validate_df, test_df, model)

val: (0.7473103289996134, 0.6151419616596224)
train: 0.606835178930439 0.4805361428138626
val: (0.520822475750004, 0.41220385770541484)
train: 0.4757892324542197 0.38006537861164646
val: (0.44208866948033193, 0.35802764813449545)
train: 0.4180096816202666 0.3426151462412771
val: (0.40410267910046516, 0.3335855035564734)
train: 0.3875334633946526 0.32308996881893765
val: (0.37868319465825706, 0.3166363542452884)
train: 0.3661391437356122 0.30695524819861153
val: (0.3604964695735768, 0.30179046293685113)
train: 0.3485468263748942 0.2879042638231379
val: (0.3402293697318645, 0.2753946540632052)
train: 0.33365334783670486 0.26922908909834187
val: (0.330631677097293, 0.2654078644965046)
train: 0.32733389363980764 0.2626523983599796
val: (0.32700851024657934, 0.2627324770601625)
train: 0.32354824948900557 0.2600798686645783
val: (0.3224022426817918, 0.25977926799171963)
train: 0.31940089727608695 0.2572212044104914
val: (0.31915272446458454, 0.25739049644503675)
train: 0.31636031402446024 0.

In [98]:
user_vec, user_bias = model.user_lut.weight.data, model.user_bias.weight.data

In [99]:
user_features = np.zeros((user_vec.numpy().shape[0], user_vec.numpy().shape[1]+1))
user_features[:, :user_vec.numpy().shape[1]] = user_vec.numpy()
user_features[:, -1] = user_bias.numpy().reshape(-1)

In [100]:
user_features

array([[-1.01259172,  0.08032684,  0.22459139],
       [ 0.03830139,  0.04006179, -0.15898089],
       [ 1.5714947 , -0.86501276, -0.0041402 ],
       ..., 
       [ 0.11534408, -0.09271911, -0.02346769],
       [ 0.49681965, -0.66163236, -0.05740545],
       [-0.15671408,  0.41284984,  0.02198376]])

In [101]:
# np.savetxt('./user_id_mapping.csv', testframe[['uid','user_id']].drop_duplicates().values)
np.savetxt('./cluster_array.csv', kmeans.labels_)

In [102]:
kmeans = KMeans(n_clusters=200, random_state=0).fit(user_features)
print(kmeans.labels_)

[127  59 114 ..., 156  93 144]


In [105]:
completed, values, ind = compute_individual_dist(testframe, True, False)

In [106]:
start = time.time()
features = np.empty((ind.shape[0], ind.shape[1]**2*ind.shape[0]))
delta_matrices_all = np.empty((ind.shape[0], ind.shape[0], ind.shape[1], ind.shape[1]))
score_matrices_all = np.empty((ind.shape[0], ind.shape[0], ind.shape[1], ind.shape[1]))
for user_index in range(values.shape[0]):
    if user_index%100==0:
        print(user_index)
    #check for full joint distribution or add a prior later
    if np.sum(ind[user_index]==0) > 0:
        continue
    #compute delta matrices with all other users where applicable
    else:
        #create a mask so that other half of tasks can be used later to find score matrix
#         mask = np.random.randint(0,2,values.shape).astype(bool)
        mask = np.ones((values.shape)).astype(bool)
        delta_matrices, t_m_i_1, cluster_img = compute_deltas(user_index, completed, values, ind, mask, False, 20)
        features[user_index,:] = cluster_img.flatten()
        delta_matrices_all[user_index,:,:,:] = delta_matrices
#         score_matrices, t_m_i_2 = compute_deltas(user_index, completed, values, ind, ~mask, True, 20)
#         score_matrices_all[user_index,:,:,:] = score_matrices
#         print(np.sum(t_m_i_1), np.sum(t_m_i_2))
#         if len(np.intersect1d(np.array(np.where(t_m_i_1==True)), np.array(np.where(t_m_i_2==True))))>0:
#             print(np.intersect1d(np.array(np.where(t_m_i_1==True)), np.array(np.where(t_m_i_2==True))))
#             print(regret(score_matrices, delta_matrices, \
#                      np.logical_and((t_m_i_1==True), (t_m_i_2==True))))
print(time.time()-start)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
653.079264163971


In [149]:
np.sum(np.isnan(features))

110825

In [107]:
from sklearn.cluster import KMeans

def kmeans_missing(X, n_clusters, max_iter=10):
    """Perform K-Means clustering on data with missing values.

    Args:
      X: An [n_samples, n_features] array of data to cluster.
      n_clusters: Number of clusters to form.
      max_iter: Maximum number of EM iterations to perform.

    Returns:
      labels: An [n_samples] vector of integer labels.
      centroids: An [n_clusters, n_features] array of cluster centroids.
      X_hat: Copy of X with the missing values filled in.
    """

    # Initialize missing values to their column means
    missing = ~np.isfinite(X)
    mu = np.nanmean(X, 0, keepdims=1)
    X_hat = np.where(missing, mu, X)

    for i in range(max_iter):
        print(i)
        if i > 0:
            # initialize KMeans with the previous set of centroids. this is much
            # faster and makes it easier to check convergence (since labels
            # won't be permuted on every iteration), but might be more prone to
            # getting stuck in local minima.
            cls = KMeans(n_clusters, init=prev_centroids)
        else:
            # do multiple random initializations in parallel
            cls = KMeans(n_clusters, n_jobs=-1)

        # perform clustering on the filled-in data
        labels = cls.fit_predict(X_hat)
        centroids = cls.cluster_centers_

        # fill in the missing values based on their cluster centroids
        X_hat[missing] = centroids[labels][missing]

        # when the labels have stopped changing then we have converged
        if i > 0 and np.all(labels == prev_labels):
            break

        prev_labels = labels
        prev_centroids = cls.cluster_centers_

    return labels, centroids, X_hat

In [108]:
labels, centroids, X_hat = kmeans_missing(features, 400, max_iter=5)
print(labels)

0
1


/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:893: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


2
[220  99 170 ..., 266 343 331]


In [109]:
def calc_cluster_matrix_dist(labels, delta_matrices_all):
#     delta_matrices_new=np.zeros((len(np.unique(labels)), len(np.unique(labels)), delta_matrices_all[0][0].shape[0], \
#                                                               delta_matrices_all[0][0].shape[1]))
    delta_matrices_new=np.empty(delta_matrices_all.shape)
    for i in np.unique(labels):
        if i%100==0:
            print(i)
        missing = ~np.isfinite(delta_matrices_all)
        mu = np.nanmean(delta_matrices_all, 0, keepdims=0)
        filled = np.where(missing, mu, delta_matrices_all)
        cluster_i = np.average(filled[labels==i], axis=0)
        for j in np.unique(labels): 
            if np.sum(np.sum(np.isnan(cluster_i), (1,2)))>0:
                print(i,j)
                missing = ~np.isfinite(deltas_used)
                mu = np.nanmean(delta_matrices_all, 1, keepdims=1)
                X_hat = np.where(missing, mu, X)
            cluster_j = np.average(cluster_i[labels==j], axis=0)
#             delta_matrices_new[i,j] = cluster_j
            delta_matrices_new[np.ix_(labels==i,labels==j)] = cluster_j
    return delta_matrices_new

In [185]:
delta_matrices_clust = calc_cluster_matrix_dist(labels, delta_matrices_all)

0
100


In [189]:
print(np.average([[pairwise_distances(score_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)]))

0.0776220916645


In [190]:
delta_matrices_clust = calc_cluster_matrix_dist(kmeans.labels_, delta_matrices_all)
print(np.average([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)]))

0
100
0.0770279382517


In [113]:
for k_ in range(2,11):
    model = Model(users, tasks, k=k_)
    user_vec, user_bias = train(train_df, validate_df, test_df, model)
    user_features = np.zeros((user_vec.numpy().shape[0], user_vec.numpy().shape[1]+1))
    user_features[:, :user_vec.numpy().shape[1]] = user_vec.numpy()
    user_features[:, -1] = user_bias.numpy().reshape(-1)
    kmeans = KMeans(n_clusters=400, random_state=0).fit(user_features)
    delta_matrices_clust = calc_cluster_matrix_dist(kmeans.labels_, delta_matrices_all)
    avg_dist = np.average([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)])
    max_dist = np.max([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)])
    print("K={} score: {}, {}".format(k_, avg_dist, max_dist))

val: 2.0777278104227674
train: 1.2116290182206695
val: 0.7037363884206135
train: 0.5415771868267919
val: 0.43095153492469124
train: 0.37284708612514905
val: 0.3362389822791087
train: 0.3094920255758393
val: 0.29511266203916064
train: 0.2803961135279256
val: 0.2742695826174684
train: 0.26502186941334716
val: 0.2624492080619879
train: 0.2560319880641963
val: 0.25516313670855634
train: 0.2503637469994149
val: 0.25037620876514394
train: 0.24657596241028473
val: 0.24706860682050535
train: 0.24392504133559736
val: 0.24468791332265538
train: 0.24199866034805284
val: 0.24291570638787846
train: 0.24055453016340203
val: 0.24155883453926144
train: 0.23944327906537716
val: 0.2404949847923139
train: 0.2385690068180709
val: 0.23964382042786905
train: 0.23786796768224575
val: 0.23895085063834187
train: 0.23729649213738355
val: 0.23837808474724526
train: 0.2368238659764394
val: 0.237898370021001
train: 0.23642797699748727
val: 0.2374918730891948
train: 0.23609257782100876
val: 0.23714382638434262
trai

val: 0.5979119211726214
train: 0.5010595408194991
val: 0.4538116255161054
train: 0.39934205657254496
val: 0.3825611433077871
train: 0.34635565289352627
val: 0.3419400636195742
train: 0.31518187334284775
val: 0.31651896121910467
train: 0.2952847477721761
val: 0.2995167080574935
train: 0.28181112939753883
val: 0.28755905959853967
train: 0.2722634799855486
val: 0.2788098085295374
train: 0.26524854139250786
val: 0.2721996359413626
train: 0.2599395658947294
val: 0.2670715384187277
train: 0.25582116521817966
val: 0.26300377728905894
train: 0.2525587221990473
val: 0.2597154946690391
train: 0.24992741627249682
val: 0.25701370432973375
train: 0.24777169049116016
val: 0.2547621452760772
train: 0.2459811378737559
val: 0.2528623085063793
train: 0.2444756289580709
val: 0.2512414721467619
train: 0.24319585078115669
val: 0.24984494856713665
train: 0.24209712377480808
val: 0.24863093145147605
train: 0.2411452577017321
val: 0.24756698048329648
train: 0.24031371610020275
val: 0.24662758518998684
train: 

In [217]:
np.sum(np.isnan(user_features))

0

In [220]:
# for k_ in range(2,3):
#     model = Model(users, tasks, k=k_)
#     user_vec, user_bias = train(train_df, validate_df, test_df, model)
#     user_features = np.empty((user_vec.numpy().shape[0], user_vec.numpy().shape[1]+1))
#     user_features[:, :user_vec.numpy().shape[1]] = user_vec.numpy()
#     user_features[:, :-1] = user_bias.numpy()
sc = KMeans(n_clusters=200, random_state=0).fit(user_features)
delta_matrices_clust = calc_cluster_matrix_dist(sc.labels_, delta_matrices_all)
avg_dist = np.average([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)])
max_dist = np.max([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)])
print("K={} score: {}, {}".format(k_, avg_dist, max_dist))

/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/numpy/core/_methods.py:116: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:248: RuntimeWarning: invalid value encountered in add
  distances += XX
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:400: RuntimeWarning: overflow encountered in square
  max_iter=max_iter, verbose=verbose)
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:400: RuntimeWarning: invalid value encountered in subtract
  max_iter=max_iter, verbose=verbose)
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:401: RuntimeWarning: overflow encountered in square
  inertia = np.sum((X - centers[labels]) ** 2, dtype=np.float64)


0
K=2 score: 0.07221198567612774, 0.7124813399972646


In [91]:
kmeans = SpectralClustering(n_clusters=200).fit(user_features)
# delta_matrices_clust = calc_cluster_matrix_dist(kmeans.labels_, delta_matrices_all)
# avg_dist = np.average([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)])
# print("K={} score: {}".format(k_, avg_dist))

In [92]:
kmeans.labels_

array([190,   1, 197,  85,   3,   1,   1, 199,   1, 190,  22,   3,   1,
       188, 192,   1, 192,   1, 188,   1, 192, 190,   1, 191,   1, 166,
         3, 188, 196, 188, 190,   1,   3,  14,   1,   1,   1, 188,  21,
         1, 133, 143,   1, 108, 177, 196,   1,   1, 197,  25,   1,   1,
         1, 188,   1, 157, 188, 193, 104, 193, 188, 190,   1,   1,   1,
       188, 188,   0, 188, 188, 188, 188, 188, 190,   1,   1,   1,  53,
         3,   1,   1,   1,   1,   3,   1,   1,   1, 188,   3, 193,   0,
         3,   1,   1,   1, 188,   1,   1,   1,   1,   1, 190,   1,   1,
         1, 127, 188, 190,   3, 188,   1, 188,   1,   1,   1, 195,   1,
       193, 188, 199,   1, 159,   1, 197,   1,   1, 188, 155, 196,   1,
       188, 132,   1,   1,  91, 193, 188, 164, 188, 141,   1,   1, 196,
         1,   1, 190,   1, 193,   1, 188,   1,   3,   1,   1, 196, 188,
        36,  92,   1,   1,  20,   1, 193, 106, 197,   1,   1,   1, 197,
         1,   1,   1, 188,   3,   0, 188, 188,   1, 188,   1,   

In [222]:
sc.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,